In [11]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
import zipfile

In [12]:
device='cpu'

In [24]:
def load_data_jay_lyrics(): 
    with zipfile.ZipFile(r'F:\study\ml\ebooks3\6\jaychou_lyrics.txt.zip') as zif:
        with zif.open('jaychou_lyrics.txt') as f:
            corpus_chars =f.read().decode('utf-8')
    corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
    corpus_chars=corpus_chars[0:10000]
    
    idx_to_char=list(set(corpus_chars))
    char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
    vocab_size=len(char_to_idx)
    corpus_indices=[char_to_idx[char] for char in corpus_chars]
    
    return corpus_indices,char_to_idx,idx_to_char,vocab_size

In [25]:
corpus_indices,char_to_idx,idx_to_char,vocab_size=load_data_jay_lyrics()

In [26]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)
rnn_layer

RNN(1027, 256)

In [27]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
Y,state_new=rnn_layer(X,state)
print(X.shape)
print(Y.shape,len(state_new),state_new[0].shape)

torch.Size([35, 2, 1027])
torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [40]:
def grad_clipping(params,theta):
    norm=torch.Tensor([0.0])
    for p in params:
        norm +=(p.grad.data **2).sum()
    norm = np.sqrt(norm.item())
    if norm > theta:
        for p in params:
            p.grad.data *= (theta/norm)

In [41]:
def one_hot(x,n_class,dtype=torch.float32):
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res

In [42]:
def to_onehot(x,n_class):
    return [one_hot(x[:,i],n_class) for i in range(x.shape[1])]

In [43]:
x=torch.Tensor([[1,2,3],[4,5,6]])
to_onehot(x,10)

[tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])]

In [44]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super().__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        self.state=None
        
    def forward(self,inputs,state):
        X=to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        # state : num_layers, batch_size, hidden_nums
        #X     :  num_steps , batch_size, vocab_size
        #Y     :  num_steps , batch_size, hidden_nums
        #output:  hidden_nums,           vocab_size
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state

In [45]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X=torch.tensor([output[-1]],device=device).view(-1,1)
        if state is not None:
            if isinstance(state,tuple):
                state=(state[0].to(device),state[1].to(device))
            else:
                state=state.to(device)
        (Y,state)=model(X,state)
        if t<len(prefix) -1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [46]:
model=RNNModel(rnn_layer,vocab_size).to(device)
predict_rnn_pytorch('分开',10,model,vocab_size,device,idx_to_char,char_to_idx)

'分开童整视理黑理疗黑理疗'

In [47]:
torch.device('cpu')

device(type='cpu')

In [48]:
corpus_indices

[676,
 230,
 927,
 81,
 111,
 541,
 554,
 676,
 230,
 125,
 897,
 38,
 43,
 472,
 405,
 819,
 554,
 676,
 230,
 125,
 897,
 392,
 426,
 820,
 72,
 1002,
 554,
 392,
 426,
 820,
 472,
 405,
 21,
 554,
 949,
 673,
 711,
 673,
 711,
 673,
 711,
 820,
 676,
 676,
 676,
 676,
 304,
 897,
 554,
 818,
 227,
 983,
 132,
 870,
 554,
 771,
 949,
 1026,
 590,
 462,
 613,
 915,
 600,
 894,
 554,
 45,
 572,
 955,
 598,
 452,
 427,
 554,
 771,
 949,
 491,
 43,
 897,
 554,
 928,
 88,
 983,
 771,
 949,
 851,
 285,
 983,
 788,
 207,
 40,
 381,
 554,
 984,
 46,
 983,
 771,
 949,
 598,
 402,
 983,
 788,
 207,
 40,
 381,
 554,
 1022,
 877,
 983,
 771,
 949,
 45,
 482,
 983,
 788,
 207,
 40,
 381,
 554,
 114,
 114,
 983,
 771,
 949,
 962,
 872,
 983,
 788,
 207,
 40,
 381,
 554,
 114,
 114,
 983,
 771,
 949,
 962,
 872,
 983,
 788,
 207,
 40,
 381,
 554,
 928,
 88,
 983,
 771,
 949,
 851,
 285,
 983,
 788,
 207,
 40,
 381,
 554,
 984,
 46,
 983,
 771,
 949,
 598,
 402,
 983,
 788,
 207,
 40,
 381,
 554,
 1

In [49]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1) // num_steps
    for i in range(epoch_size):
        i=i+num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [50]:
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,num_epochs,num_steps,lr,
                                 clipping_theta,batch_size,pred_period,pred_len,prefixes):
    loss=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_consecutive(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if state is not None:
                if isinstance(state,tuple):
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
            (output,state)=model(X,state)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(output,y.long())
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(),clipping_theta)
            optimizer.step()
            l_sum +=l.item() * y.shape[0]
            n+=y.shape[0]
            
            try:
                perplexity=math.exp(l_sum/n)
            except OverflowError:
                perplexity=float('inf')
            if (epoch +1) % pred_period ==0:
                print('epoch %d ,perplexity %f,time %.2f sec' % (epoch +1 ,perplexity,time.time()-start))
                
                for prefix in prefixes:
                    print('- ',predict_rnn_pytorch(prefix,pred_len,model,vocab_size,device,idx_to_char,char_to_idx))
                    

In [51]:
num_steps=35
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                              corpus_indices, idx_to_char, char_to_idx,
                              num_epochs, num_steps, lr, clipping_theta,
                              batch_size, pred_period, pred_len, prefixes)

epoch 50 ,perplexity 1.380694,time 0.11 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.313758,time 0.28 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.260181,time 0.42 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.228829,time 0.56 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.211987,time 0.73 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.197511,time 0.87 sec
-  分开始想像 爸和妈当年的模样 说著你 一片云掉落在我面前 捏成你的形状 随风跟著我 一口一口吃掉落 我面
-  不分开  我跟了你 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 50 ,perplexity 1.187435,time 1.02 sec
-  分开始想像 爸和妈当年